In [ ]:
import pandas as pd
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1RxjFKAfwKRvJ-w4_miQKZV9COwi5RhiN'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('mannualData'+'.csv')  
mannualData = pd.read_csv('mannualData'+'.csv')

In [ ]:
x = [[ "එය ARG1" , "සංකීර්ණ ARG2" , "වන be.01" , "අතර O" , "එය O" , "යුක්ති O" , "සහගත O" , "නොවේ O" , ". O" ],[ "එය ARG1" , "සංකීර්ණ complicate.01" , "වන O" , "අතර O" , "එය O" , "යුක්ති O" , "සහගත O" , "නොවේ O" , ". O" ],[ "එය O" , "සංකීර්ණ O" , "වන O" , "අතර O" , "එය ARG1" , "යුක්ති justify.01" , "සහගත justify.01" , "නොවේ ARGM-NEG" , ". O" ]]

def makeLists (mannualData = x):
  noOfPredsInMannualDta = len(mannualData)
  newMannualData = []
  for i in range(0, noOfPredsInMannualDta):
    mannualSeq = mannualData[i] #[ "එය ARG1" , "සංකීර්ණ ARG2" , "වන be.01" , "අතර O" , "එය O" , "යුක්ති O" , "සහගත O" , "නොවේ O" , ". O" ]
    newMannualSeq = [] #[ ["එය","ARG1"] , ["සංකීර්ණ","ARG2"] , ["වන","be.01"] , ["අතර","O"] , ["එය","O"] , ["යුක්ති","O"] , ["සහගත","O"] , ["නොවේ","O"] , [".","O"] ]
    for j in mannualSeq:
      a = j.strip()
      newMannualSeq.append(a.split())
    newMannualData.append(newMannualSeq)
  return newMannualData

In [ ]:
a = makeLists()
print(a)

[[['එය', 'ARG1'], ['සංකීර්ණ', 'ARG2'], ['වන', 'be.01'], ['අතර', 'O'], ['එය', 'O'], ['යුක්ති', 'O'], ['සහගත', 'O'], ['නොවේ', 'O'], ['.', 'O']], [['එය', 'ARG1'], ['සංකීර්ණ', 'complicate.01'], ['වන', 'O'], ['අතර', 'O'], ['එය', 'O'], ['යුක්ති', 'O'], ['සහගත', 'O'], ['නොවේ', 'O'], ['.', 'O']], [['එය', 'O'], ['සංකීර්ණ', 'O'], ['වන', 'O'], ['අතර', 'O'], ['එය', 'ARG1'], ['යුක්ති', 'justify.01'], ['සහගත', 'justify.01'], ['නොවේ', 'ARGM-NEG'], ['.', 'O']]]


In [ ]:
def replaceAtsWithComma(Resultslist): #replace @@@s With Commas
  res = []
  for i in Resultslist:
    if isinstance(i, list):
      insideL = []
      for j in i:
        x = j.replace('@@@', ',')
        x = x.strip()
        insideL.append(x)
      res.append(insideL)
  # if len(res) == 1:  
  #   return res[0]
  return res

In [ ]:
def stringToList(ini_list): #Convert a string to a list
    # print(ini_list)
    res = ini_list.replace('],[','], [').replace('] ,[','], [').replace('] , [','], [').strip()[1:-1].split("], [")
    res = [[i] for i in res]
    # print(res)
    newRes = []
    for i in res:
      res = i[0].replace(', ,', ', @@@')  #to handle exceptions when "," is in the sentence
      newRes.append(res.strip('][').split(' , ')) 
    newRes = replaceAtsWithComma(newRes)
    # print(newRes)
    return newRes

In [ ]:
def get_chunks(seq):
    chunks = []
    chunk_type, chunk_start = None, None
    for i, tok in enumerate(seq):
        if tok[1] == 'O' and chunk_type is not None:
            chunk = (chunk_type, chunk_start, i)
            chunks.append(chunk)
            chunk_type, chunk_start = None, None

        elif tok[1] != 'O' :
            tok_chunk = tok[1]
            if chunk_type is None:
                chunk_type, chunk_start = tok_chunk, i
            elif tok_chunk != chunk_type:
                chunk = (chunk_type, chunk_start, i)
                chunks.append(chunk)
                chunk_type, chunk_start = tok_chunk, i
        else:
            pass

    if chunk_type is not None:
        chunk = (chunk_type, chunk_start, len(seq))
        chunks.append(chunk)

    return chunks

In [ ]:
def comparableSeq(sentence):
  s = [get_chunks(seq) for seq in sentence]
  flat_list = [item for sublist in s for item in sublist]
  return flat_list

In [ ]:
accs = []
correct_preds = 0
total_preds = 0
total_correct = 0

for index, row in mannualData.iterrows():
# for i in range(1):
  if row['manual annotation'] != '[ ]':
    s1 = stringToList(row['manual annotation'])
    for u in s1:
      if u[-1] != '. O':
        u.append('. O')
  else:
    s1 = []
  if row['predicted annotation'] != '[ ]':
    s2 = stringToList(row['predicted annotation'])
    for u in s2:
      if u[-1] != '. O':
        u.append('. O')
  else:
    s2 = []
  # s1 = [['එසේ ARGM-DIS', 'වුවද ARGM-DIS', ', O', 'තීරණය ARG1', 'දැනටමත් ARGM-TMP', 'සම්මත adopt.01', 'කර adopt.01', 'ඇත have.01', '. O']]
  # print(s1)
  sentence1 = makeLists(s1) #[[['එය', 'ARG0'], ['තවත්', 'ARG1'], ['බොහෝ', 'O'], ['දේ', 'O'], ['කළ', 'do.02'], ['යුතුව', 'O'], ['තිබුණි', 'O'], ['.', 'O']], [['එය', 'O'], ['තවත්', 'O'], ['බොහෝ', 'O'], ['දේ', 'O'], ['කළ', 'O'], ['යුතුව', 'O'], ['තිබුණි', 'have.01'], ['.', 'O']]]
  # print(sentence1)
  sentence2 = makeLists(s2)
  # print(sentence2)
  CSentence1 = comparableSeq(sentence1)
  # print(CSentence1)
  CSentence2 = comparableSeq(sentence2)
  # print(CSentence2)
  accs += [a==b for (a, b) in zip(CSentence1, CSentence2)]
  CSentence1 = set(CSentence1)
  CSentence2 = set(CSentence2)
  correct_preds += len(CSentence1 & CSentence2)
  total_preds   += len(CSentence2)
  total_correct += len(CSentence1)

p   = correct_preds / total_preds if correct_preds > 0 else 0
r   = correct_preds / total_correct if correct_preds > 0 else 0
f1  = 2 * p * r / (p + r) if correct_preds > 0 else 0
acc = np.mean(accs)

print("accuracy " + str(100*acc))
print("p " + str(p))
print("r " + str(r))
print("f1 " + str(f1))

accuracy 47.56394640682095
p 0.59357780733422
r 0.47995083730219695
f1 0.5307509344206592


In [ ]:
# For projector tool
# accuracy 78.84978540772532
# p 0.8311425682507584
# r 0.757720079889384
# f1 0.792734871011814